In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import  silhouette_score
from datetime import datetime

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sn
pio.renderers.default = "vscode"

In [2]:
#### uber-raw-data-apr14
data_01 = pd.read_csv("uber-raw-data-apr14.csv")
data_01.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [3]:
# Convertir la chaîne de caractères en objet datetime
data_01['Date/Time'] = pd.to_datetime(data_01['Date/Time'])

# Extraire l'année, le mois, le jour et le jour de la semaine
data_01['Date'] = data_01['Date/Time'].dt.date
data_01['Heure'] = data_01['Date/Time'].dt.time

In [4]:
# Drop la colonne Date
data_01 = data_01.drop(columns = ['Date/Time'])
data_01.head()

,Lat,Lon,Base,Date,Heure
0,40.7690,-73.9549,B02512,2014-04-01,00:11:00
1,40.7267,-74.0345,B02512,2014-04-01,00:17:00
2,40.7316,-73.9873,B02512,2014-04-01,00:21:00
3,40.7588,-73.9776,B02512,2014-04-01,00:28:00
4,40.7594,-73.9722,B02512,2014-04-01,00:33:00


In [5]:
data_01_R = data_01["Date"].value_counts()
data_01_R.sort_index()

Date
2014-04-01    14546
2014-04-02    17474
2014-04-03    20701
2014-04-04    26714
2014-04-05    19521
2014-04-06    13445
2014-04-07    19550
2014-04-08    16188
2014-04-09    16843
2014-04-10    20041
2014-04-11    20420
2014-04-12    18170
2014-04-13    12112
2014-04-14    12674
2014-04-15    20641
2014-04-16    17717
2014-04-17    20973
2014-04-18    18074
2014-04-19    14602
2014-04-20    11017
2014-04-21    13162
2014-04-22    16975
2014-04-23    20346
2014-04-24    23352
2014-04-25    25095
2014-04-26    24925
2014-04-27    14677
2014-04-28    15475
2014-04-29    22835
2014-04-30    36251
Name: count, dtype: int64

In [6]:
data_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564516 entries, 0 to 564515
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Lat     564516 non-null  float64
 1   Lon     564516 non-null  float64
 2   Base    564516 non-null  object 
 3   Date    564516 non-null  object 
 4   Heure   564516 non-null  object 
dtypes: float64(2), object(3)
memory usage: 21.5+ MB


In [7]:
data_01['Date'] = pd.to_datetime(data_01['Date'])

In [8]:
def assign_letter(day):
    if day < datetime(2014, 4, 5):
        return 'A'
    elif day < datetime(2014, 4, 10):
        return 'B'
    elif day < datetime(2014, 4, 15):
        return 'C'
    elif day < datetime(2014, 4, 20):
        return 'D'
    elif day < datetime(2014, 4, 25):
        return 'E'
    elif day < datetime(2014, 4, 30):
        return 'F'            
    else:
        return 'G'

In [9]:
# Ajout de la colonne "Cat" avec les lettres attribuées
data_01['Cat_Day'] = data_01['Date'].apply(lambda x: assign_letter(x.day))

TypeError: '<' not supported between instances of 'int' and 'datetime.datetime'

In [ ]:
# Voir les variables avec des données vident
data_01.isnull().sum()

In [ ]:
# Voir le nouveau dataframe "data_01"
data_01.head()

In [ ]:
# Fonction pour attribuer une lettre en fonction de l'heure
def assign_letter(hour):
    if hour < 6:
        return 'A'
    elif hour < 12:
        return 'B'
    elif hour < 18:
        return 'C'
    else:
        return 'D'

In [ ]:
# Ajout de la colonne "Cat" avec les lettres attribuées
data_01['Cat'] = data_01['Heure'].apply(lambda x: assign_letter(x.hour))

In [ ]:
# Voir les variables avec des données vident
data_01.isnull().sum()

In [ ]:
# Voir le nouveau dataframe "data_01"
data_01.head()

In [ ]:
# Voir la distribution de la nouvelle colonne "Cat"
data = data_01['Cat'].value_counts()
data

In [ ]:
# Voir la distribution de la nouvelle colonne "Cat" en graphique
### "A" 00H00 / 06H00  -- "B" 06H00 / 12H00  -- "C" 12H00 / 18H00  -- "D" 18H00 / 00H00
plt.figure()
plt.pie(data.values, labels=data.index,
       autopct='%1.1f%%',
       shadow=True,
       startangle=90,
       radius=1.5
       )
plt.axis('equal')
plt.title('Share of time slot (Cat)')
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

In [ ]:
# Type des colonnes
data_01.info()

In [ ]:
# Convertissez la colonne d'heures en format datetime
data_01['Heure'] = pd.to_datetime(data_01['Heure'], format='%H:%M:%S')

In [ ]:
data_01.info()

In [ ]:
## Connaitre le nombre d'id unique (personne) par genre.
id_unique = data_01.groupby('Cat').value_counts()
id_unique

In [ ]:
# Créer des nouveaux DataFrames pour chaque valeur unique de la colonne "Cat"
nouveaux_dfs = {}
for cat, group in data_01.groupby('Cat'):
    nouveaux_dfs[cat] = group.drop(columns='Cat')

# Afficher les nouveaux DataFrames
for cat, df_cat in nouveaux_dfs.items():
    print(f"DataFrame pour Catégorie '{cat}':")
    print(df_cat)
    print()

In [ ]:
# Création de nouveau dataframe par "Cat"
data_01_A = nouveaux_dfs['A']
data_01_B = nouveaux_dfs['B']
data_01_C = nouveaux_dfs['C']
data_01_D = nouveaux_dfs['D']

In [ ]:
# Visualisation des dataframes
print("Dataframe data_01_A")
print(data_01_A)


print("Dataframe data_01_B")
print(data_01_B)


print("Dataframe data_01_C")
print(data_01_C)


print("Dataframe data_01_D")
print(data_01_D)

In [ ]:
#### Traitement de la "Cat" "A" 00H00 / 06H00

numeric_features = [0, 1] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# Création du transformer pour les variables catégorielles
categorical_features = [2,3,4] # Positions des colonnes catégorielles dans X
categorical_transformer = OneHotEncoder(drop='first')

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(data_01_A.head())
X_A = preprocessor.fit_transform(data_01_A) # fit_transform !!
print('...Terminé.')
print(X_A[0:5, :])
print()

In [ ]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss_A =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i, n_init = "auto")
    kmeans.fit(X_A)
    wcss_A.append(kmeans.inertia_)
    
print(wcss_A)

In [ ]:
fig = px.line(x = range(2,11), y = wcss_A)
fig.show()

In [ ]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score_A = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i, n_init = "auto")
    kmeans.fit(X_A)
    s_score_A.append(silhouette_score(X_A, kmeans.predict(X_A)))

print(s_score_A)

In [ ]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score_A)
fig.show()

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 3, n_init = "auto")
kmeans.fit(X_A)

In [ ]:
data_01_A.loc[:,'Cluster_KMeans'] = kmeans.predict(X_A)
data_01_A.head()

In [ ]:
# On utilise scatter_mapbox pour visualiser notre graphique
fig = px.scatter_mapbox(data_frame=data_01_A, lat='Lat', lon='Lon', color='Cluster_KMeans', mapbox_style="carto-positron")
fig.show()

In [ ]:
#### uber-raw-data-may14
data_02 = pd.read_csv("uber-raw-data-may14.csv")
data_02.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_02['Date/Time'] = pd.to_datetime(data_02['Date/Time'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_02['date'] = data_02['Date/Time'].dt.date
data_02['heure'] = data_02['Date/Time'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_02['date'] = data_02['Date/Time'].dt.date
data_02['heure'] = data_02['Date/Time'].dt.time

In [ ]:
# Drop la colonne Date
data_02 = data_02.drop(columns = ['Date/Time'])
data_02.head()

In [ ]:
#### uber-raw-data-jun14
data_03 = pd.read_csv("uber-raw-data-jun14.csv")
data_03.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_03['Date/Time'] = pd.to_datetime(data_03['Date/Time'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_03['date'] = data_03['Date/Time'].dt.date
data_03['heure'] = data_03['Date/Time'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_03['date'] = data_03['Date/Time'].dt.date
data_03['heure'] = data_03['Date/Time'].dt.time

In [ ]:
# Drop la colonne Date
data_03 = data_03.drop(columns = ['Date/Time'])
data_03.head()

In [ ]:
#### uber-raw-data-jul14
data_04 = pd.read_csv("uber-raw-data-jul14.csv")
data_04.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_04['Date/Time'] = pd.to_datetime(data_04['Date/Time'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_04['date'] = data_04['Date/Time'].dt.date
data_04['heure'] = data_04['Date/Time'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_04['date'] = data_04['Date/Time'].dt.date
data_04['heure'] = data_04['Date/Time'].dt.time

In [ ]:
# Drop la colonne Date
data_04 = data_04.drop(columns = ['Date/Time'])
data_04.head()

In [ ]:
#### uber-raw-data-jul14
data_05 = pd.read_csv("uber-raw-data-aug14.csv")
data_05.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_05['Date/Time'] = pd.to_datetime(data_05['Date/Time'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_05['date'] = data_05['Date/Time'].dt.date
data_05['heure'] = data_05['Date/Time'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_05['date'] = data_05['Date/Time'].dt.date
data_05['heure'] = data_05['Date/Time'].dt.time

In [ ]:
# Drop la colonne Date
data_05 = data_05.drop(columns = ['Date/Time'])
data_05.head()

In [ ]:
#### uber-raw-data-sep14
data_06 = pd.read_csv("uber-raw-data-sep14.csv")
data_06.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_06['Date/Time'] = pd.to_datetime(data_06['Date/Time'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_06['date'] = data_06['Date/Time'].dt.date
data_06['heure'] = data_06['Date/Time'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_06['date'] = data_06['Date/Time'].dt.date
data_06['heure'] = data_06['Date/Time'].dt.time

In [ ]:
# Drop la colonne Date
data_06 = data_06.drop(columns = ['Date/Time'])
data_06.head()

In [ ]:
#### uber-raw-data-janjune-15
data_07 = pd.read_csv("uber-raw-data-janjune-15.csv")
data_07.head()

In [ ]:
# Convertir la chaîne de caractères en objet datetime
data_07['Pickup_date'] = pd.to_datetime(data_07['Pickup_date'])
# Extraire l'année, le mois, le jour et le jour de la semaine
data_07['date'] = data_07['Pickup_date'].dt.date
data_07['heure'] = data_07['Pickup_date'].dt.time

In [ ]:
# Extraire l'année, le mois, le jour et le jour de la semaine
data_07['date'] = data_07['Pickup_date'].dt.date
data_07['heure'] = data_07['Pickup_date'].dt.time

In [ ]:
# Drop la colonne Date
data_07 = data_07.drop(columns = ['Pickup_date'])
data_07.head()